#  GTFSデータをpydeckで可視化する

## 概要

- GTFSのデータを [deck.gl](https://deck.gl/#/)を Python で利用するためのパッケージ pydeckを使って可視化を行う
 - [pydeck: Unlocking deck.gl for use in Python](https://medium.com/vis-gl/pydeck-unlocking-deck-gl-for-use-in-python-ce891532f986)

背景地図にmapboxを利用するため、環境変数にアクセスキーを設定しておく

```
export MAPBOX_API_KEY=<your access key>
```

### 利用したGTFSデータ

- [北海道拓殖バス オープンデータ](https://www.takubus.com/%E3%82%AA%E3%83%BC%E3%83%97%E3%83%B3%E3%83%87%E3%83%BC%E3%82%BF/)

### 必要なパッケージのインポート

In [1]:
import os
import zipfile
import pandas as pd

import pydeck

### ディレクトリの構成

dataフォルダにはGTFSのZIPファイルを格納しておく

In [2]:
data_path = "data"
tmp_path = "tmp"
output_path = "output"

### GTFSのZIPファイルから必要なファイルを展開

In [3]:
# GTFSファイルの指定
gtfs_file = "GTFS_Takushoku_Obihiro.zip"

# 必要なファイルを展開しておく
with zipfile.ZipFile(os.path.join(os.getcwd(), f"{data_path}/{gtfs_file}")) as _zip:
    _zip.extract("stops.txt", tmp_path)
    _zip.extract("stop_times.txt", tmp_path)
    _zip.extract("shapes.txt", tmp_path)
    _zip.extract("trips.txt", tmp_path)
    _zip.extract("routes.txt", tmp_path)

### 停留所の座標データの読み込み

In [4]:
df_stops = pd.read_csv(os.path.join(os.getcwd(), f"{tmp_path}/stops.txt"))
df_stops = df_stops.dropna(axis='columns', how='all') # 列方向、全てNaNなら除去
df_stops['stop_z'] = 0 # 後でGeoJSONに書き出すため、ダミーの標高値を入れておく
df_stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,location_type,platform_code,stop_z
0,1000_01,NaN,帯広駅バスターミナル,NaN,42.918091,143.203767,1000_01,0,NaN,0
1,1000_02,2番のりば,帯広駅バスターミナル,ターミナル2番のりば,42.918120,143.203642,1000_02,0,2,0
2,1000_03,3番のりば,帯広駅バスターミナル,ターミナル3番のりば,42.918246,143.203402,1000_03,0,3,0
3,1000_04,4番のりば,帯広駅バスターミナル,ターミナル4番のりば,42.918326,143.203227,1000_04,0,4,0
4,1000_06,6番のりば,帯広駅バスターミナル,ターミナル5番のりば,42.918635,143.203425,1000_06,0,6,0
...,...,...,...,...,...,...,...,...,...,...
923,9843_01,NaN,役場御影支所前,バス停ポールは支所玄関前（上下線共通）,42.945692,142.942203,9843_01,0,NaN,0
924,9844_01,NaN,世代間交流センター前,バス停ポールは施設玄関前（上下線共通）,42.944764,142.939803,9844_01,0,NaN,0
925,9845_01,NaN,羽帯 国道38号線沿い《清水町コミュニティバス》,NaN,42.959234,142.908407,9845_01,0,NaN,0
926,9845_02,NaN,羽帯 国道38号線沿い《清水町コミュニティバス》,NaN,42.959453,142.907377,9845_02,0,NaN,0


### 経路の形状座標データ


In [5]:
df_shapes= pd.read_csv(os.path.join(os.getcwd(), f"{tmp_path}/shapes.txt"))
df_shapes = df_shapes.dropna(axis='columns', how='all') # 列方向、全てNaNなら除去
df_shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,1011,42.918246,143.203402,1
1,1011,42.918359,143.203279,2
2,1011,42.918540,143.203468,3
3,1011,42.918590,143.204152,4
4,1011,42.919110,143.204036,5
...,...,...,...,...
14680,8614,43.014639,142.881222,145
14681,8614,43.013985,142.880170,146
14682,8614,43.013661,142.880550,147
14683,8614,43.013555,142.880371,148


## pydeckで描画する

In [8]:
import pydeck

layer_stops = pydeck.Layer(
    'ScatterplotLayer',
    df_stops,
    id='gtfs_stops',
    get_position=['stop_lon', 'stop_lat'],
    get_radius=30,
    get_fill_color=[200, 100, 100, 255],
    auto_highlight=True,
    pickable=True
)

layer_shapes = pydeck.Layer(
    'ScatterplotLayer',
    df_shapes,
    id='gtfs_shapes',
    get_position=['shape_pt_lon', 'shape_pt_lat'],
    get_radius=10,
    get_fill_color=[250,250,80,255],
    auto_highlight=True,
    pickable=True
)

view_state = pydeck.ViewState(
    longitude=143.10,
    latitude=43.024,
    zoom=10,
    min_zoom=5,
    max_zoom=15,
    pitch=30,
    bearing=0)

# viewport = pydeck.data_utils.compute_view(df_stops[['stop_lon', 'stop_lat']])

r = pydeck.Deck(
    layers=[layer_stops, layer_shapes],
    views=[pydeck.View(type="MapView", controller=True)],
    map_style="mapbox://styles/mapbox/dark-v9",
    mapbox_key=None,
    initial_view_state=view_state,
    width="100%",
    height=600,
)

r.show()

DeckGLWidget(height=600, json_input='{"initialViewState": {"bearing": 0, "latitude": 43.024, "longitude": 143.…

In [ ]:
# HTMLファイルを出力しブラウザで表示する
r.to_html(f"{output_path}/gtfs_stops.html", open_browser=True, notebook_display=False)